In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

base_dir = r'E:\Khóa luận\Data\Phase 2'
files = {
    '250500': '250500.csv',
    '500500': '500500.csv',
    '750500': '750500.csv'
}
export_dir = r'E:\Khóa luận\Exports\rawdata_phase2'
os.makedirs(export_dir, exist_ok=True)

for label, fname in files.items():
    print(f"\n===== Đang xử lý file: {fname} (nồng độ {label}) =====")
    file_path = os.path.join(base_dir, fname)
    df = pd.read_csv(file_path, encoding='utf-8', sep=';')
    print(f"Số dòng dữ liệu: {len(df)}")
    print(f"Số cột: {len(df.columns)}")
    
    # Phát hiện các cột toàn NaN hoặc toàn 0
    all_nan_cols = df.columns[df.isna().all()].tolist()
    all_zero_cols = df.columns[(df == 0).all()].tolist()
    print(f"Cột toàn NaN: {all_nan_cols}")
    print(f"Cột toàn 0: {all_zero_cols}")

    # Thống kê số lượng giá trị NaN và 0 trên từng cột
    nan_counts = df.isna().sum()
    zero_counts = (df == 0).sum()
    print("Top 5 cột nhiều NaN nhất:")
    print(nan_counts.sort_values(ascending=False).head())
    print("Top 5 cột nhiều giá trị 0 nhất:")
    print(zero_counts.sort_values(ascending=False).head())

    # Kiểm tra các cột có số lượng giá trị duy nhất thấp (dễ nghi ngờ lỗi)
    nunique = df.nunique(dropna=True)
    few_unique_cols = nunique[nunique <= 2].index.tolist()
    print(f"Cột có <=2 giá trị duy nhất (có thể là biến nhị phân hoặc lỗi): {few_unique_cols}")

    # Danh sách các biến số (loại trừ cột thời gian, id...)
    numerical_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col]) and col not in all_nan_cols and col not in all_zero_cols]
    if 'Timestamp' in numerical_cols: numerical_cols.remove('Timestamp')
    if 'Date time' in numerical_cols: numerical_cols.remove('Date time')

    # Chuyển đổi ngày giờ nếu có
    if 'Date time' in df.columns:
        df['Date time'] = pd.to_datetime(df['Date time'], errors='coerce')

    # ========== VẼ TOÀN BỘ BIẾN SỐ TRÊN 1 ẢNH ==========
    # 1. Violinplot
    if len(numerical_cols) > 0:
        plt.figure(figsize=(max(10, len(numerical_cols)*1.2), 6))
        sns.violinplot(data=df[numerical_cols], inner='box', cut=0)
        plt.title(f'Violinplot tất cả biến số - Nồng độ {label}')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(export_dir, f'violinplot_all_{label}.png'))
        plt.close()
    else:
        print(f"[CẢNH BÁO] Không có biến số nào để vẽ violinplot cho nồng độ {label}.")

    # 2. Boxplot
    if len(numerical_cols) > 0:
        plt.figure(figsize=(max(10, len(numerical_cols)*1.2), 6))
        sns.boxplot(data=df[numerical_cols])
        plt.title(f'Boxplot tất cả biến số - Nồng độ {label}')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(export_dir, f'boxplot_all_{label}.png'))
        plt.close()
    else:
        print(f"[CẢNH BÁO] Không có biến số nào để vẽ boxplot cho nồng độ {label}.")

    # 3. Histogram (subplots)
    if len(numerical_cols) > 0:
        n_cols = 4
        n_rows = int(np.ceil(len(numerical_cols)/n_cols))
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*4, n_rows*3))
        axes = axes.flatten()
        for i, col in enumerate(numerical_cols):
            sns.histplot(df[col], bins=30, kde=True, ax=axes[i], color='steelblue')
            axes[i].set_title(col)
        for j in range(i+1, len(axes)):
            axes[j].axis('off')
        plt.suptitle(f'Histogram các biến số - Nồng độ {label}', fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.97])
        plt.savefig(os.path.join(export_dir, f'histogram_all_{label}.png'))
        plt.close()
    else:
        print(f"[CẢNH BÁO] Không có biến số nào để vẽ histogram cho nồng độ {label}.")

    # ========== CÁC TRỰC QUAN KHÁC GIÚP CHẨN ĐOÁN ==========
    # Heatmap tương quan
    if len(numerical_cols) > 1:
        plt.figure(figsize=(max(8, len(numerical_cols)), max(6, len(numerical_cols)*0.6)))
        corr = df[numerical_cols].corr()
        sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
        plt.title(f'Heatmap ma trận tương quan - Nồng độ {label}')
        plt.tight_layout()
        plt.savefig(os.path.join(export_dir, f'corr_heatmap_{label}.png'))
        plt.close()
    else:
        print(f"[CẢNH BÁO] Không đủ biến số để vẽ heatmap tương quan cho nồng độ {label}.")

    # Biểu đồ missing value (barplot)
    if len(numerical_cols) > 0:
        missing_df = pd.DataFrame({
            'missing': nan_counts,
            'zero': zero_counts
        })
        missing_df['total'] = missing_df['missing'] + missing_df['zero']
        missing_df = missing_df.loc[numerical_cols].sort_values('total', ascending=False)
        plt.figure(figsize=(max(8, len(numerical_cols)*0.7), 5))
        missing_df[['missing', 'zero']].plot(kind='bar', stacked=True, ax=plt.gca(), color=['#FFA07A', '#87CEFA'])
        plt.title(f'Số lượng giá trị thiếu và bằng 0 trên từng biến số - Nồng độ {label}')
        plt.ylabel('Số lượng')
        plt.xlabel('Tên biến')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(export_dir, f'missing_and_zero_{label}.png'))
        plt.close()
    else:
        print(f"[CẢNH BÁO] Không có biến số nào để vẽ missing/zero barplot cho nồng độ {label}.")

    # Time series plot cho 3 biến đầu tiên (nếu có cột thời gian)
    if 'Date time' in df.columns and len(numerical_cols) > 0:
        for col in numerical_cols[:3]:
            if df[col].nunique(dropna=True) < 2 or df[col].dropna().empty:
                continue
            plt.figure(figsize=(12, 4))
            plt.plot(df['Date time'], df[col], label=col)
            plt.title(f'Chuỗi thời gian của {col} - Nồng độ {label}')
            plt.xlabel('Thời gian')
            plt.ylabel(col)
            plt.tight_layout()
            plt.savefig(os.path.join(export_dir, f'timeseries_{col}_{label}.png'))
            plt.close()
    elif 'Date time' in df.columns:
        print(f"[CẢNH BÁO] Không có biến số nào hợp lệ để vẽ time series cho nồng độ {label}.")

print(f"\nĐã lưu toàn bộ biểu đồ và thông tin chẩn đoán vào thư mục: {export_dir}")


===== Đang xử lý file: 250500.csv (nồng độ 250500) =====
Số dòng dữ liệu: 11839
Số cột: 15
Cột toàn NaN: ['HCHOmg/m3', 'TVOCppm']
Cột toàn 0: ['H2S_concentration']
Top 5 cột nhiều NaN nhất:
HCHOmg/m3        11839
TVOCppm          11839
H2S_boardtemp        8
NH3_boardtemp        8
Timestamp            0
dtype: int64
Top 5 cột nhiều giá trị 0 nhất:
H2S_concentration    11839
CH4ppm               10663
ccsTVOC               4998
Timestamp                0
Date time                0
dtype: int64
Cột có <=2 giá trị duy nhất (có thể là biến nhị phân hoặc lỗi): ['H2S_concentration', 'HCHOmg/m3', 'TVOCppm']

===== Đang xử lý file: 500500.csv (nồng độ 500500) =====
Số dòng dữ liệu: 8646
Số cột: 15
Cột toàn NaN: ['HCHOmg/m3', 'TVOCppm']
Cột toàn 0: ['ccsTVOC', 'H2S_concentration']
Top 5 cột nhiều NaN nhất:
HCHOmg/m3    8646
TVOCppm      8646
Timestamp       0
Date time       0
ccseCO2         0
dtype: int64
Top 5 cột nhiều giá trị 0 nhất:
ccsTVOC              8646
H2S_concentration    8646
CH4